In [1]:
###############################################################################
"""
Python Glacier Evolution Model "PyGEM" V1.0
Prepared by David Rounce with support from Regine Hock.
This work was funded under the NASA HiMAT project (INSERT PROJECT #).

PyGEM is an open source glacier evolution model written in python.  Model
details come from Radic et al. (2013), Bliss et al. (2014), and Huss and Hock
(2015).
"""
###############################################################################
# This is the main script that provides the architecture and framework for all
# of the model runs. All input data is included in a separate module called
# pygem_input.py. It is recommended to not make any changes to this file unless
# you are a PyGEM developer and making changes to the model architecture.
#
#========== IMPORT PACKAGES ==================================================
# Various packages are used to provide the proper architecture and framework
# for the calculations used in this script. Some packages (e.g., datetime) are
# included in order to speed of calculations and simplify code
import pandas as pd
import numpy as np
from datetime import datetime
import os # os is used with re to find name matches
import re # see os
import xarray as xr
#========= IMPORT MODEL INPUTS ===============================================
from pygem_input import *
    # import all data
    # pygem_input.py contains all the input data
#========== IMPORT FUNCTIONS FROM MODULES ====================================
import pygemfxns_modelsetup as modelsetup
import pygemfxns_climate as climate
import pygemfxns_massbalance as massbalance

In [2]:
#========== OTHER STEPS ======================================================
# Other steps:
# Geodetic mass balance file path
# ???[Create input script full of option list]???
#       - Select regions: (Option 1 - default regions from RGI inventory)
#                         (Option 2 - custom regions)
#       - Select climate data:
#           - climate data source/file name
#           - climate data resolution
#           - sampling scheme (nearest neighbor, etc.)
#       - Model run type: (Option 1 - calibration)
#                         (Option 2 - simulation)
#

#========== LIST OF OUTPUT ===================================================
# Create a list of outputs (.csv or .txt files) that the user can choose from
# depending on what they are using the model for or want to see:
#    1. Time series of all variables for each grid point (x,y,z) of each glacier
#       and every time step
#       (--> comparing with point balances)
#    2. Time series of all variables for elevation bands if spatial
#       discretization is elevation bands (x,y,z) of each glacier and every time
#       step
#       (--> comparing with profiles)
#    3. as above but all variables averaged over current single glacier surfaces
#       (--> comparing with geodetic mass balances of glaciers)
#    4. Time series of all variables averaged over region
#       (--> comparison to GRACE)
#    5. Time series of seasonal balance for individual glaciers
#       (--> comparison to WGMS data)
# Also develop output log file, i.e., file that states input parameters,
# date of model run, model options selected, and any errors that may have come
# up (e.g., precipitation corrected because negative value, etc.)

#========== MODEL RUN DETAILS ================================================
# The model is run through a series of steps:
#   > Step 01: Region/Glaciers Selection
#              The user needs to define the region/glaciers that will be used in
#              the model run.  The user has the option of choosing the standard
#              RGI regions or defining their own.
#   > Step 02: Model Time Frame
#              The user should consider the time step and duration of the model
#              run along with any calibration product and/or model spinup that
#              may be included as well.
#   > Step 03: Climate Data
#              The user has the option to choose the type of climate data being
#              used in the model run, and how that data will be downscaled to
#              the glacier and bins.
#   > Step 04: Glacier Evolution
#              The heart of the model is the glacier evolution, which includes
#              calculating the specific mass balance, the surface type, and any
#              changes to the size of the glacier (glacier dynamics). The user
#              has many options for how this aspect of the model is run.
#   > Others: model output? model input?

In [3]:
#----- STEP ONE: MODEL REGION/GLACIERS ---------------------------------------
# Step one involves the selection of the regions and glaciers used in the model.
# Regions/glacier included in the model run will be defined using the Randolph
#   Glacier inventory.  For more information, see:
#     https://www.glims.org/RGI/ (RGI Consortium, 2017)
# In step one, the model will:
#   > select glaciers included in model run

# Select glaciers that are included in the model run
# Glacier Selection Options:
#   > 1 (default) - enter numbers associated with RGI V6.0 and select
#                   glaciers accordingly
#   > 2 - glaciers/regions selected via shapefile
#   > 3 - glaciers/regions selected via new table (other inventory)
#
if option_glacier_selection == 1:
    main_glac_rgi = modelsetup.selectglaciersrgitable()
elif option_glacier_selection == 2:
    # OPTION 2: CUSTOMIZE REGIONS USING A SHAPEFILE that specifies the
    #           various regions according to the RGI IDs, i.e., add an
    #           additional column to the RGI table.
    # ??? [INSERT CODE FOR IMPORTING A SHAPEFILE] ???
    #   (1) import shapefile with custom boundaries, (2) grab the RGIIDs
    #   of glaciers that are in these boundaries, (3) perform calibration
    #   using these alternative boundaries that may (or may not) be more
    #   representative of regional processes/climate
    #   Note: this is really only important for calibration purposes and
    #         post-processing when you want to show results over specific
    #         regions.
    # Development Note: if create another method for selecting glaciers,
    #                   make sure that update way to select glacier
    #                   hypsometry as well.
    print('\n\tMODEL ERROR (selectglaciersrgi): this option to use'
          '\n\tshapefiles to select glaciers has not been coded yet.'
          '\n\tPlease choose an option that exists. Exiting model run.\n')
    exit()
else:
    # Should add options to make regions consistent with Brun et al. (2017),
    # which used ASTER DEMs to get mass balance of 92% of the HMA glaciers.
    print('\n\tModel Error (selectglaciersrgi): please choose an option'
          '\n\tthat exists for selecting glaciers. Exiting model run.\n')
    exit()


This study is only focusing on glaciers ['03473', '03733'] in region [15].
The 'select_rgi_glaciers' function has finished.


In [7]:
#----- STEP TWO: ADDITIONAL MODEL SETUP --------------------------------------
# Step two runs more functions related to the model setup. This section has been
#   separated from the selection of the model region/glaciers in order to
#   keep the input organized and easy to read.
# In step two, the model will:
#   > select glacier hypsometry
#   > define the model time frame
#   > define the initial surface type

# Glacier hypsometry
# main_glac_hyps = modelsetup.hypsometryglaciers(main_glac_rgi)
    # Note: need to adjust this hypsometry into separate functions such that it
    #       is easier to follow.
# AUTOMATE THIS TO LOAD THEM IN INSTEAD OF CHOOSING THEM
# main_glac_hyps = pd.read_csv(hyps_filepath + 'RGI_13_area_test20170905.csv')
ds = pd.read_csv(hyps_filepath + 'bands_10m_DRR/area_15_Huss_SouthAsiaEast_10m.csv')
print(ds.head())

             RGI-ID  Cont_range  25  35  45  55  65  75  85  95  ...   8725  \
0  RGIv6.0.15-00001           1 -99 -99 -99 -99 -99 -99 -99 -99  ...  -99.0   
1  RGIv6.0.15-00002           1 -99 -99 -99 -99 -99 -99 -99 -99  ...  -99.0   
2  RGIv6.0.15-00003           1 -99 -99 -99 -99 -99 -99 -99 -99  ...  -99.0   
3  RGIv6.0.15-00004           1 -99 -99 -99 -99 -99 -99 -99 -99  ...  -99.0   
4  RGIv6.0.15-00005           0 -99 -99 -99 -99 -99 -99 -99 -99  ...  -99.0   

   8735  8745  8755  8765  8775  8785  8795  8805  8815  
0   -99 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0  
1   -99 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0  
2   -99 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0  
3   -99 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0  
4   -99 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0  

[5 rows x 882 columns]


In [64]:
# Select glaciers based on 01Index value from main_glac_rgi table
glac_hyps_table = pd.DataFrame()
for glacier in range(len(main_glac_rgi)):
    if glac_hyps_table.empty:
        glac_hyps_table = ds.loc[main_glac_rgi.loc[glacier,'O1Index']]
    else:
        glac_hyps_table = pd.concat([glac_hyps_table, ds.loc[main_glac_rgi.loc[glacier,'O1Index']]], axis=1)
glac_hyps_table = glac_hyps_table.transpose()
print(glac_hyps_table.head())

                RGI-ID Cont_range   25   35   45   55   65   75   85   95  \
3472  RGIv6.0.15-03473          1  -99  -99  -99  -99  -99  -99  -99  -99   
3732  RGIv6.0.15-03733          1  -99  -99  -99  -99  -99  -99  -99  -99   

     ...  8725 8735 8745 8755 8765 8775 8785 8795 8805 8815  
3472 ...   -99  -99  -99  -99  -99  -99  -99  -99  -99  -99  
3732 ...   -99  -99  -99  -99  -99  -99  -99  -99  -99  -99  

[2 rows x 882 columns]


In [65]:
# Clean up table and re-index
# Reset index to be GlacNo
glac_hyps_table.reset_index(drop=True, inplace=True)
glac_hyps_table.index.name = indexname
glac_hyps_table.drop(['RGI-ID','Cont_range'], axis=1, inplace=True)
print(glac_hyps_table)

         25   35   45   55   65   75   85   95  105  115 ...  8725 8735 8745  \
GlacNo                                                   ...                   
0       -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 ...   -99  -99  -99   
1       -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 ...   -99  -99  -99   

       8755 8765 8775 8785 8795 8805 8815  
GlacNo                                     
0       -99  -99  -99  -99  -99  -99  -99  
1       -99  -99  -99  -99  -99  -99  -99  

[2 rows x 880 columns]


In [5]:
# Glacier initial ice thickness
# AUTOMATE THIS TO LOAD THEM IN INSTEAD OF CHOOSING THEM
main_glac_icethickness = pd.read_csv(hyps_filepath +
                                     'RGI_13_thickness_test20170905.csv')
# ADD OPTION FOR VOLUME-AREA SCALING

# Glacier total initial volume
main_glac_rgi['Volume'] = (
    (main_glac_hyps * main_glac_icethickness/1000).sum(axis=1))
    # volume [km3] = area[km2] * thickness[m] * (1 [km] / 1000 [m])

print(main_glac_rgi.head(),'\n')

        O1Index           RGIId     CenLon     CenLat  O1Region  O2Region  \
GlacNo                                                                      
0          3472  RGI60-15.03473  86.715912  28.089468        15         2   
1          3732  RGI60-15.03733  86.903244  27.974716        15         2   

          Area  Zmin  Zmax  Zmed  Slope  Aspect   Lmax     Volume  
GlacNo                                                             
0       61.054  4702  8181  5815   16.5     180  18285  84.527037  
1       19.097  4926  7870  5568   17.5     262  15396   0.533249   



In [6]:
# Model time frame
#   Set up table of dates. These dates are used as column headers for many other
#   variables in the model run, so it's important to be an initial step.
dates_table, start_date, end_date = modelsetup.datesmodelrun(option_leapyear)
print(dates_table.head())

The 'datesmodelrun' function has finished.
               date  year  month  daysinmonth
timestep                                     
0        2000-01-01  2000      1           31
1        2000-02-01  2000      2           29
2        2000-03-01  2000      3           31
3        2000-04-01  2000      4           30
4        2000-05-01  2000      5           31


In [7]:
# PUT IN OPTION TO USE LEAP YEAR AND WATER YEAR

# Water year for northern hemisphere using USGS definition (October 1 - September 30th),
# e.g., water year for 2000 is from October 1, 1999 - September 30, 2000
dates_table['year_water'] = dates_table['year']
for step in range(len(dates_table)):
    if dates_table.loc[step, 'month'] >= 10:
        dates_table.loc[step, 'year_water'] = dates_table.loc[step, 'year'] + 1

# Develops note: MAKE ADJUSTMENTS FOR LATITUDE THRESHOLDS, ETC. LIKE VALENTINA DID IF DESIRED
        
print(dates_table.head(15))

# Valentina's code:
# % m1 is the starting month of the mass balance year (start of winter). It depends on the latitude of the glacier.
# % m2 = 13-m1; and is used to index Prec and Temp
# param.latitudethresh = 75;  %different definition of start of winter/summer below and above this latitude
# if (lat > param.latitudethresh)
#     m1=9; m2=4;
# elseif (lat >= 0 && lat <= param.latitudethresh)
#     m1=10; m2=3;
# elseif (lat < -param.latitudethresh)
#     m1=3; m2=10;
# else %-75 < lat < 0
#     m1=4; m2=9;
# end

               date  year  month  daysinmonth  year_water
timestep                                                 
0        2000-01-01  2000      1           31        2000
1        2000-02-01  2000      2           29        2000
2        2000-03-01  2000      3           31        2000
3        2000-04-01  2000      4           30        2000
4        2000-05-01  2000      5           31        2000
5        2000-06-01  2000      6           30        2000
6        2000-07-01  2000      7           31        2000
7        2000-08-01  2000      8           31        2000
8        2000-09-01  2000      9           30        2000
9        2000-10-01  2000     10           31        2001
10       2000-11-01  2000     11           30        2001
11       2000-12-01  2000     12           31        2001
12       2001-01-01  2001      1           31        2001
13       2001-02-01  2001      2           28        2001
14       2001-03-01  2001      3           31        2001


In [8]:
# Initial surface type
main_glac_surftypeinit = modelsetup.surfacetypeglacinitial(
                                                option_surfacetype_initial,
                                                option_surfacetype_firn,
                                                option_surfacetype_debris,
                                                main_glac_rgi,
                                                main_glac_hyps)

The 'initialsurfacetype' function has finished.


In [11]:
# Mass balance estimates from High-res DEMs
mb_filepath = '../DEMs/hma_mb_20170717_1846.csv'
# mb_filepath = os.path.dirname(__file__) + '/../DEMs/hma_mb_20170717_1846.csv'
main_glac_mb_raw = pd.read_csv(mb_filepath)
print(main_glac_mb_raw.head())

   # glacnum           x          y    z_med    z_p16    z_p84  mb_mwea  \
0   14.19942  -927804.01  -35168.38  7496.07  7415.89  7650.54     1.02   
1   14.23897 -1157128.32  106023.79  7408.39  7353.91  7467.04     1.74   
2   15.05141   -44297.11 -837269.98  7349.13  7026.94  7473.72     0.64   
3   14.19948  -929689.48  -36435.04  7249.72  6932.15  7544.95     0.53   
4   14.23883 -1157892.34  106248.27  7193.10  7071.63  7298.66     1.12   

   area_km2       t1      t2     dt  
0      2.82  2000.11  2015.0  14.89  
1      0.10  2000.11  2015.0  14.89  
2      1.57  2000.11  2015.0  14.89  
3      2.29  2000.11  2015.0  14.89  
4      0.38  2000.11  2015.0  14.89  


In [9]:
#----- STEP THREE: CLIMATE DATA ----------------------------------------------
# Step three imports the climate data that will be used in the model run.
# Provide options for the selection and downscaling of the data
#    - default: nearest neighbor
#    - alternatives: weighted methods
#      (note: prior to any weighting, lapse rates/biases need to be applied)
# Important to consider the following:
#    - time period of the calibration data or model run
#    - type of model (DDF, EBM, etc.) will dictate meteorological data needed
#   Datasets:
#     Past: Default: ERA reanslysis?
#           Alternatives: COAWST (S.Nichols), NCEP/NCAR(?), others?
#                         automatic weather stations
#     Future: Default: GCMs (see glacierMIP project emails to download data)
#             Alternatives: COAWST (S.Nichols), others?
#

In [10]:
# In step three, the model will:
#   > import meteorological data
#   > select meteorological data for each glacier based on specified option
#       default: nearest neighbor
if option_gcm_downscale == 1:
    # OPTION 1 (default): NEAREST NEIGHBOR
    # Thoughts on 2017/08/21:
    #   > Pre-processing functions should be coded and added after the initial
    #     import such that the initial values can be printed if necessary.
    #   > Data imported here is monthly, i.e., it is 1 value per month. If the
    #     data is going to be subsampled to a daily resolution in order to
    #     estimate melt in areas with low monthly mean temperature as is done in
    #     Huss and Hock (2015), then those calculations should be performed in
    #     the ablation section.
    gcm_glac_temp = climate.importGCMvarnearestneighbor(gcm_temp_varname,
                                                        main_glac_rgi,
                                                        dates_table)
        # gcm nearest neighbor time series for each glacier with GlacNo index
        # rows = # of glaciers, cols = length of time series
    gcm_glac_prec = climate.importGCMvarnearestneighbor(gcm_prec_varname,
                                                        main_glac_rgi,
                                                        dates_table)
        # gcm nearest neighbor time series for each glacier with GlacNo index
        # rows = # of glaciers, cols = length of time series
    gcm_glac_elev = climate.importGCMfxnearestneighbor(gcm_elev_varname,
                                                       main_glac_rgi)
        # gcm nearest neighbor surface altitude for each glacier with GlacNo
        # index, rows = # of glaciers, cols = 1 (Series)
else:
    print('\n\tModel Error: please choose an option that exists for'
          '\n\tdownscaling climate data. Exiting model run now.\n')
    exit() # if you have an error, exit the model run

print(gcm_glac_temp.head())

The 'importGCMvarnearestneighbor' fxn for 'tas' has finished.
The 'importGCMvarnearestneighbor' fxn for 'pr' has finished.
The 'importGCMfxnearestneighbor' fxn for 'orog' has finished.
          2000-01   2000-02   2000-03   2000-04   2000-05    2000-06  \
GlacNo                                                                 
0      -10.502808 -7.882294 -0.942657  6.147675  11.90567  11.852539   
1       -9.147339 -7.106903 -0.382111  6.294159  10.59317  11.092773   

          2000-07    2000-08   2000-09   2000-10    ...      2100-03  \
GlacNo                                              ...                
0       16.923706  15.013214  7.489838  3.064972    ...     3.170441   
1       16.359253  15.079620  7.021088  3.662628    ...     2.879425   

         2100-04    2100-05    2100-06    2100-07    2100-08    2100-09  \
GlacNo                                                                    
0       8.817596  15.482788  18.429291  22.631256  22.814758  18.546356   
1       8.43

In [11]:
# use xarray to select climate data
# xarray is built to emulate the netcdf framework, is built on top of pandas,
# and is lazy - it won't load in the data until it is used.
# Therefore, instead of importing everything and then selecting the data as needed,
# xarray will only import the data that is being used.  It should greatly speed up 
# computational time.

variablename = 'tas'
# Import netcdf file
filefull = gcm_filepath_var + variablename + gcm_filename_var
data = xr.open_mfdataset(filefull)
# mfdataset uses dask to speed up processing
print(data['tas'])

<xarray.DataArray 'tas' (time: 2772, lat: 96, lon: 192)>
dask.array<open_dataset-b3f237a6ce742e793cb70511c7663e2btas, shape=(2772, 96, 192), dtype=float64, chunksize=(2772, 96, 192)>
Coordinates:
  * time     (time) datetime64[ns] 1870-01-15T12:00:00 1870-02-15 ...
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 -81.13 -79.27 -77.41 ...
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 9.375 11.25 13.12 15.0 ...
Attributes:
    units:          K
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature


In [16]:
data.coords

Coordinates:
  * time     (time) datetime64[ns] 1870-01-15T12:00:00 1870-02-15 ...
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 -81.13 -79.27 -77.41 ...
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 9.375 11.25 13.12 15.0 ...

In [17]:
data.sel(time='2000-01-15')

<xarray.Dataset>
Dimensions:  (lat: 96, lon: 192, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-15T12:00:00
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 -81.13 -79.27 -77.41 ...
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 9.375 11.25 13.12 15.0 ...
Data variables:
    tas      (time, lat, lon) float64 240.1 240.1 240.0 240.1 240.0 240.0 ...
Attributes:
    source_model:        MPI-ESM-LR
    source_ensemble:     r1i1p1
    source_experiment:   historical,rcp85
    source_institution:  Max Planck Institute for Meteorology
    source_contact:      cmip5-mpi-esm@dkrz.de
    source_files:        tas_Amon_MPI-ESM-LR_historical_r1i1p1_185001-200512....
    source_md5:          14ea33cdaec9d54cae0f986255b4b21b,82e3ca65b6338ff918f...
    freq:                monthly
    interpolation_grid:  native
    contact:             cmip5-archive@env.ethz.ch
    modifications:       nothing

In [18]:
data.sel(lon=180.0)

<xarray.Dataset>
Dimensions:  (lat: 96, time: 2772)
Coordinates:
  * time     (time) datetime64[ns] 1870-01-15T12:00:00 1870-02-15 ...
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 -81.13 -79.27 -77.41 ...
    lon      float64 180.0
Data variables:
    tas      (time, lat) float64 237.3 245.1 254.0 258.2 257.8 258.1 264.8 ...
Attributes:
    source_model:        MPI-ESM-LR
    source_ensemble:     r1i1p1
    source_experiment:   historical,rcp85
    source_institution:  Max Planck Institute for Meteorology
    source_contact:      cmip5-mpi-esm@dkrz.de
    source_files:        tas_Amon_MPI-ESM-LR_historical_r1i1p1_185001-200512....
    source_md5:          14ea33cdaec9d54cae0f986255b4b21b,82e3ca65b6338ff918f...
    freq:                monthly
    interpolation_grid:  native
    contact:             cmip5-archive@env.ethz.ch
    modifications:       nothing

In [19]:
data['tas'].sel(lon=180.0)

<xarray.DataArray 'tas' (time: 2772, lat: 96)>
dask.array<getitem, shape=(2772, 96), dtype=float64, chunksize=(2772, 96)>
Coordinates:
  * time     (time) datetime64[ns] 1870-01-15T12:00:00 1870-02-15 ...
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 -81.13 -79.27 -77.41 ...
    lon      float64 180.0
Attributes:
    units:          K
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature

In [ ]:
data['tas'].sel(time='2015lon=180.0)